# Minimal Neural Plasticity Test

This notebook provides a minimal test of the neural plasticity module functionality.

Version: v0.0.60 (2025-04-20 09:15:27)

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoModelForCausalLM, AutoTokenizer

# Import the modular neural plasticity API
from utils.neural_plasticity import NeuralPlasticity
from utils.neural_plasticity import PruningStrategy, PruningMode

# Get environment information
env_info = NeuralPlasticity.get_environment_info()
print("Environment Information:")
print(f"- Platform: {env_info['platform']}")
print(f"- Apple Silicon: {env_info['is_apple_silicon']}")
print(f"- GPU Available: {env_info['has_gpu']}")
print(f"- Device: {env_info['device']}")

# Define device - respect environment detection
device = env_info['device']


In [ ]:
# Load a small model for testing
model_name = "distilgpt2"  # Small model for faster testing
print(f"Loading model: {model_name}")

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Get model structure information
num_layers, num_heads = NeuralPlasticity.detect_attention_heads(model)
print(f"Model structure: {num_layers} layers with {num_heads} heads each")

In [ ]:
# Prepare a small test input
test_text = "This is a test of the neural plasticity system."
inputs = tokenizer(test_text, return_tensors="pt").to(device)
print(f"Input shape: {inputs['input_ids'].shape}")

In [ ]:
# Analyze attention patterns
attention_data = NeuralPlasticity.analyze_attention_patterns(
    model=model,
    input_ids=inputs['input_ids'],
    attention_mask=inputs['attention_mask']
)

# Extract data
attention_tensors = attention_data['attention_tensors']
entropy_values = attention_data['entropy_values']

# Print basic statistics
print(f"Number of attention tensors: {len(attention_tensors)}")
print(f"Attention tensor shape: {attention_tensors[0].shape}")
print(f"Number of entropy layers: {len(entropy_values)}")
print(f"Entropy shape: {entropy_values[0].shape}")

# Visualize entropy
from utils.neural_plasticity import visualize_head_entropy

plt.figure(figsize=(10, 6))
entropy_fig = visualize_head_entropy(
    entropy_values=entropy_values,
    title="Attention Entropy (Test)",
    min_value=0.0,
    annotate=True
)
plt.show()

In [ ]:
# Test gradient calculation - simplified version for minimal test
def minimal_gradient_test(model, inputs):
    model.train()  # Set to training mode for gradient calculation
    
    # Forward pass with gradient tracking
    outputs = model(**inputs)
    loss = outputs.loss
    
    # Backward pass
    loss.backward()
    
    # Get model structure
    num_layers, num_heads = NeuralPlasticity.detect_attention_heads(model)
    
    # Extract gradient norms (simplified)
    from utils.neural_plasticity import extract_head_gradient
    grad_norms = torch.zeros((num_layers, num_heads), device='cpu')
    
    for layer_idx in range(num_layers):
        for head_idx in range(num_heads):
            grad = extract_head_gradient(model, layer_idx, head_idx)
            if grad is not None:
                grad_norms[layer_idx, head_idx] = grad.norm().item()
    
    # Reset gradients
    model.zero_grad()
    model.eval()  # Set back to eval mode
    
    return grad_norms

# Calculate gradients
grad_norms = minimal_gradient_test(model, inputs)
print(f"Gradient norms shape: {grad_norms.shape}")
print(f"Max gradient norm: {grad_norms.max().item():.4f}")
print(f"Min gradient norm: {grad_norms.min().item():.4f}")

# Visualize gradients
from utils.neural_plasticity import visualize_head_gradients

plt.figure(figsize=(10, 6))
grad_fig = visualize_head_gradients(
    grad_norm_values=grad_norms,
    title="Gradient Norms (Test)"
)
plt.show()

In [ ]:
# Generate pruning mask
from utils.neural_plasticity import generate_pruning_mask

# Set pruning parameters
prune_percent = 0.2  # Prune 20% of heads
strategy = PruningStrategy.COMBINED  # Use combined strategy

# Generate mask
pruning_mask = generate_pruning_mask(
    grad_norm_values=grad_norms,
    entropy_values=entropy_values[0],  # Use first layer's entropy
    prune_percent=prune_percent,
    strategy=strategy
)

# Visualize pruning decisions
from utils.neural_plasticity import visualize_pruning_decisions

plt.figure(figsize=(10, 6))
pruning_fig = visualize_pruning_decisions(
    grad_norm_values=grad_norms,
    pruning_mask=pruning_mask,
    title=f"Pruning Decisions ({strategy}, {prune_percent*100:.0f}%)"
)
plt.show()

# Count pruned heads
total_heads = pruning_mask.numel()
pruned_count = pruning_mask.sum().item()
print(f"Pruning {pruned_count} out of {total_heads} heads ({pruned_count/total_heads*100:.1f}%)")

In [ ]:
# Apply pruning
from utils.neural_plasticity import apply_pruning_mask

# Apply pruning mask
pruned_heads = apply_pruning_mask(
    model=model,
    pruning_mask=pruning_mask,
    mode="zero_weights"
)

print(f"Pruned {len(pruned_heads)} heads:")
for layer, head in pruned_heads[:10]:  # Show first 10
    print(f"  Layer {layer}, Head {head}")
    
if len(pruned_heads) > 10:
    print(f"  ... and {len(pruned_heads) - 10} more")

In [ ]:
# Test text generation with pruned model
prompt = "Once upon a time"
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

# Generate text
with torch.no_grad():
    output = model.generate(
        input_ids=input_ids,
        max_length=30,
        do_sample=True,
        top_k=50,
        top_p=0.95
    )

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"Prompt: {prompt}")
print(f"Generated (pruned model): {generated_text}")

In [ ]:
# Final verification - all tests passed
print("✅ All neural plasticity module tests completed successfully!")
print(f"Tested {num_layers} layers with {num_heads} heads each")
print(f"Pruned {len(pruned_heads)} heads ({len(pruned_heads)/total_heads*100:.1f}%)")
print("Model is still functional after pruning.")